In [84]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance

In [85]:
train = pd.read_csv('dataset/development_dataset.csv')
test = pd.read_csv('dataset/leaderboard_dataset.csv')

In [86]:
train['VAR14'] = train['VAR14'].replace('.',1)
train['VAR21'] = train['VAR21'].replace({'Low','Medium','High'},(0,1,2))

In [87]:
train['VAR14']=train['VAR14'].astype(str).astype(int)
train['VAR21']=train['VAR21'].astype(str).astype(int)

In [88]:
train['VARS1'] =(train['VAR7'] - train['VAR16']).abs()
test['VARS1'] = (test['VAR7'] - test['VAR16']).abs()

In [89]:
columns = ['VAR15','VAR18','VAR7','VAR1']
train = train.drop(columns, axis=1)
test = test.drop(columns,axis=1)

In [90]:
X = train.drop(['VAR21'],axis=1)
Y = train['VAR21']

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=25)

In [92]:
D_train = xgb.DMatrix(X, label=Y)
D_test = xgb.DMatrix(X_test,label= Y_test)

In [93]:
param = {
    'eta': 0.1, 
    'max_depth': 9,  
    'objective': 'multi:softmax',
    'min child wt' :7,
    'gamma': 5,
    'num_class': 3} 

In [ ]:
model = xgb.train(param,D_train, 250)

In [73]:
preds = model.predict(D_test)

In [74]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, preds, labels=[0,1,2])

array([[  93,  566,  125],
       [  10, 3147,  443],
       [  12, 1307, 1097]], dtype=int64)

In [75]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, preds)

0.6377941176470588

In [76]:
test['VAR14'] = test['VAR14'].replace('.',1)
test['VAR14']=test['VAR14'].astype(str).astype(int)

In [77]:
D_test1 = xgb.DMatrix(test)

In [78]:
prediction = model.predict(D_test1)

In [81]:
output = pd.DataFrame()
output['VAR1'] = test['VAR1']
output['VAR21']=prediction

In [82]:
output['VAR21'] = output['VAR21'].replace((0,1,2),{'Low','Medium','High'})

In [83]:
output.to_csv(r'Neil_hanna_IITKGP_1.csv', header = None, index=False)